In [17]:
from datetime import datetime, timezone, timedelta

from database.market import Market
from processor.processor import Processor as p
from modeler_strats.universal_modeler import UniversalModeler
from datetime import datetime
from fund.fund import Fund

from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
import pandas as pd

from returns.products import Products
from processor.processor import Processor as p
market = Market()

parameter = {
    "value":True
    ,"ceiling":True
    ,"classification":False
    ,"rank":False
    ,"short":False
    ,"risk":True
    ,"market_return":1.15
    ,"buy_day":1
    ,"sell_day":5
}



start = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")

portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)

portfolio_ii = APortfolio(pricer_list.WEEKLY_CRYPTO_SPECULATION
                          ,classifier_list.NONE
                          ,ranker_list.NONE)

portfolios = []
portfolios.append(portfolio)
portfolios.append(portfolio_ii)

fund = Fund(portfolios,start,end,end)
fund.initialize_portfolios()
fund.initialize_backtesters()
modeler_strat = UniversalModeler()

day_to_deploy = parameter["buy_day"]
hour_to_deploy = 11
new_york_date = datetime.now(tz=timezone(offset=timedelta(hours=-4)))
year = new_york_date.year
week = new_york_date.isocalendar()[1]

market.cloud_connect()
sp500 = market.retrieve("sp500").rename(columns={"Symbol":"ticker"})
tyields = Products.tyields(market.retrieve("tyields"))
bench = Products.spy_bench(market.retrieve("spy"))
for portfolio in fund.portfolios:
    try:
        pricer_class = portfolio.pricer_class
        tickers = sp500["ticker"].unique() if pricer_class.asset_class.value == "stocks" else ["BTC"]
        positions = 20 if pricer_class.asset_class.value == "stocks" else 1
        training_sets = []
        for ticker in tickers:
            try:
                prices = market.retrieve_ticker_prices(pricer_class.asset_class.value,ticker)
                prices = p.column_date_processing(prices)
                ticker_data = pricer_class.training_set(ticker,prices,True)
                training_sets.append(ticker_data)
            except Exception as e:
                print(ticker,str(e))
                continue
        data = pd.concat(training_sets)
        training_data = data.dropna().copy().sort_values(["year","week"])
        market.disconnect()
        # making predictions
        pricer_class.db.cloud_connect()
        pricer_class.db.drop("predictions")
        prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
        models = pricer_class.db.retrieve("models")
        stuff = modeler_strat.recommend(models,prediction_set,pricer_class.factors)
        stuff = stuff.rename(columns={"prediction":f"price_prediction"})
        stuff = pricer_class.sim_processor(stuff)
        relevant_columns = list(set(list(stuff.columns)) - set(pricer_class.factors))
        pricer_class.db.store("predictions",stuff[relevant_columns])
        pricer_class.db.disconnect()

        # making recs
        market.cloud_connect()
        returns = portfolio.create_returns(market,bench,True)
        market.disconnect()

        sim = stuff[relevant_columns].copy().drop("adjclose",axis=1)
        merged = portfolio.merge_sim_returns(sim,returns)
        merged = merged.sort_values(["year","week","day"]).dropna()
        stuff = portfolio.backtester.recommendation(merged.copy(),parameter,tyields)

        rec = stuff[0]
        rec_filtered = rec[(rec["week"]==week)]
        trades = rec_filtered.head(positions).merge(sp500[["ticker","Security","GICS Sector"]],on="ticker")
        final = trades[["year","week","ticker","Security","GICS Sector","weekly_delta","weekly_delta_sign"]]

#         if final.index.size > 0:
#             portfolio.db.cloud_connect()
#             portfolio.db.store("recs",final)
#             portfolio.db.disconnect()

    except Exception as e:
            portfolio.db.cloud_connect()
            portfolio.db.store("errors",pd.DataFrame([{"date":str(datetime.now()),"error":str(e)}]))
            portfolio.db.disconnect()


In [18]:
final

,year,week,ticker,Security,GICS Sector,weekly_delta,weekly_delta_sign
0,2023,26,NVR,"NVR, Inc.",Consumer Discretionary,0.258458,1.0
1,2023,26,BKR,Baker Hughes,Energy,0.055661,1.0
2,2023,26,MOH,Molina Healthcare,Health Care,0.040245,1.0
3,2023,26,VTR,Ventas,Real Estate,0.035915,1.0
4,2023,26,GWW,W. W. Grainger,Industrials,0.035655,1.0
5,2023,26,CE,Celanese,Materials,0.035343,1.0
6,2023,26,WELL,Welltower,Real Estate,0.033605,1.0
7,2023,26,RF,Regions Financial Corporation,Financials,0.033000,1.0
8,2023,26,PCAR,Paccar,Industrials,0.029213,1.0
9,2023,26,XRAY,Dentsply Sirona,Health Care,0.027774,1.0
